# Packages

In [263]:
# Data wrangling
import pandas as pd
import polars as pl
import polars.selectors as cs
import numpy as np

# Visualisation
import plotnine as pn
import matplotlib.pyplot as plt
from mizani.formatters import comma_format, custom_format, currency_format, percent_format
from IPython.display import clear_output, display
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rc
import plotly.express as px

# Utils
import os
from tqdm.notebook import tqdm
import itertools
import yaml
import warnings
import time
import holidays
import pickle
import datetime

# Modelling
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer,
)
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_absolute_percentage_error,
    root_mean_squared_error,
)
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.feature_selection import VarianceThreshold

import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.layers import ReLU, LeakyReLU

rc('text', usetex=False)

jama_colour = [
    "#374e55",
    "#df8f44",
    "#00a1d5",
    "#b24745",
    "#79af97",
    "#6a6599",
    "#80796b",
]

pd.set_option("display.max.columns", 500)
pd.set_option("display.max.columns", 500)


theme_academic = pn.theme(
    text=pn.element_text(family="Latin Modern Roman"),
    plot_title=pn.element_text(weight="bold", size=14, ha="center"),
    legend_text=pn.element_text(size=9),  # Smaller font for legend items
    panel_background=pn.element_rect(fill="white"),  # Clean white background
    panel_border=pn.element_rect(color="grey", size=0.5),
    axis_ticks=pn.element_line(color="grey"),
    panel_grid_major=pn.element_line(color="grey", size=0.1, alpha=0.3),
    panel_grid_minor=pn.element_line(color="grey", size=0.1, alpha=0.3),
    legend_background=pn.element_rect(fill="white", color=None),
    legend_key=pn.element_rect(fill="white", color=None),
    plot_margin=0.02,
    figure_size=(6, 4),  # Set default figure size (width, height in inches)
)

%matplotlib inline

# Loading the data

In [197]:
df = pl.read_csv(
    "../0_data/preprocessed/df_final_reduced.csv", try_parse_dates=True
).filter(pl.col("datetime") >= pd.Timestamp("2021-09-01 00:00"))

df.head()

datetime,kWh,Zurich_shortwave_radiation,Zurich_soil_temperature_7_to_28cm
datetime[μs],f64,f64,f64
2021-09-01 00:00:00,5.1633e6,0.0,16.042
2021-09-01 01:00:00,5.0777e6,0.0,15.892
2021-09-01 02:00:00,4.9315e6,0.0,16.042
2021-09-01 03:00:00,4.7878e6,0.0,15.942
2021-09-01 04:00:00,4.7038e6,0.0,15.792001


In [198]:
df.shape

(26304, 4)

# Missing values

Just forward fill for now.

In [199]:
df = df.fill_null(strategy="forward")

# Ensure 1 hour gaps

- and availability of full days (to walk in 24 hour steps)

In [200]:
(df["datetime"] - df["datetime"].shift(1)).value_counts()

datetime,count
duration[μs],u32
null,1
1h,26303


In [201]:
df["datetime"].min()

datetime.datetime(2021, 9, 1, 0, 0)

In [202]:
df["datetime"].max()

datetime.datetime(2024, 8, 31, 23, 0)

# Calendar Features

In [203]:
df = df.with_columns(
    day_of_month=pl.col("datetime").dt.day(),
    day_of_year=pl.col("datetime").dt.ordinal_day(),
    day_of_week=pl.col("datetime").dt.weekday(),
    month=pl.col("datetime").dt.month(),
    hour=pl.col("datetime").dt.hour(),
    year=pl.col("datetime").dt.year(),
)

# Holidays

In [204]:
# Define the region (Canton of Berne) and the country (Switzerland)
country = "CH"
prov = "ZH"

# Create a list of the regional holidays for the canton of Berne
regional_holidays = holidays.CH(
    years=df["datetime"].dt.year().unique().to_list(), prov=prov
)

In [205]:
holiday_df = pl.DataFrame(
    {
        "holiday_name": list(regional_holidays.values()),
        "holiday_date": list(regional_holidays.keys()),
    }
).sort("holiday_date")

holiday_df.head()

holiday_name,holiday_date
str,date
"""Neujahrestag""",2021-01-01
"""Berchtoldstag""",2021-01-02
"""Karfreitag""",2021-04-02
"""Ostermontag""",2021-04-05
"""Tag der Arbeit""",2021-05-01


In [206]:
df = (
    df.with_columns(date=pl.col("datetime").dt.date())
    .join(holiday_df, how="left", left_on="date", right_on="holiday_date")
    .drop("date")
    .with_columns(holiday_name=pl.col("holiday_name").fill_null("no_holiday"))
)

df.head()

datetime,kWh,Zurich_shortwave_radiation,Zurich_soil_temperature_7_to_28cm,day_of_month,day_of_year,day_of_week,month,hour,year,holiday_name
datetime[μs],f64,f64,f64,i8,i16,i8,i8,i8,i32,str
2021-09-01 00:00:00,5.1633e6,0.0,16.042,1,244,3,9,0,2021,"""no_holiday"""
2021-09-01 01:00:00,5.0777e6,0.0,15.892,1,244,3,9,1,2021,"""no_holiday"""
2021-09-01 02:00:00,4.9315e6,0.0,16.042,1,244,3,9,2,2021,"""no_holiday"""
2021-09-01 03:00:00,4.7878e6,0.0,15.942,1,244,3,9,3,2021,"""no_holiday"""
2021-09-01 04:00:00,4.7038e6,0.0,15.792001,1,244,3,9,4,2021,"""no_holiday"""


# Cyclical Encoding

In [207]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


def encode_cyclically(column_name, periodicity, table):
    # Create sin and cos encoding
    table = table.with_columns(
        sin_transformer(periodicity)
        .fit_transform(table[column_name])
        .alias(f"{column_name}_sin")
    )

    table = table.with_columns(
        cos_transformer(periodicity)
        .fit_transform(table[column_name])
        .alias(f"{column_name}_cos")
    )
    # Drop the old column
    table = table.drop(column_name)

    return table

In [208]:
# Dictionary with column name and calendar periodicity
calendar_features = {
    "day_of_month": 31,
    "day_of_year": 365,
    "day_of_week": 7,
    "month": 12,
    "hour": 24,
}

for column_name, periodicity in calendar_features.items():
    df = encode_cyclically(column_name, periodicity, df)

# sklearn Pipeline

In [209]:
cat_cols = ["holiday_name"]

num_cols = df.select(
    cs.contains(
        "soil_temperature_7_to_28cm",
        "shortwave_radiation",
    )
).columns + ["year"]

manual_cols = df.select(pl.selectors.contains("_cos", "_sin", "is_")).columns

In [210]:
df.drop(manual_cols + cat_cols + num_cols)

datetime,kWh
datetime[μs],f64
2021-09-01 00:00:00,5.1633e6
2021-09-01 01:00:00,5.0777e6
2021-09-01 02:00:00,4.9315e6
2021-09-01 03:00:00,4.7878e6
2021-09-01 04:00:00,4.7038e6
…,…
2024-08-31 19:00:00,5.3614e6
2024-08-31 20:00:00,5.3775e6
2024-08-31 21:00:00,5.2130e6


In [211]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)

In [212]:
categorical_transformer = Pipeline(
    steps=[
        (
            "encoder",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        ),
    ]
)

In [213]:
column_transformer = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, num_cols),
        ("categorical", categorical_transformer, cat_cols),
    ],
    remainder="passthrough",
)

In [214]:
preprocessor = Pipeline(
    steps=[
        ("column_transformer", column_transformer),
        (
            "variance_threshold",
            VarianceThreshold(threshold=0.0),
        ),  # Drops constant columns after transformations
    ]
)

# Wide Data Format

Start by preprocessing the data in hourly frequency:

In [215]:
df_train = df.filter(
    (pl.col("datetime") >= pl.datetime(2021, 9, 1, 0))
    & (pl.col("datetime") <= pl.datetime(2022, 8, 31, 23))
).to_pandas()

df_val = df.filter(
    (pl.col("datetime") >= pl.datetime(2022, 9, 1, 0))
    & (pl.col("datetime") <= pl.datetime(2023, 8, 31, 23))
).to_pandas()

In [216]:
X_train = df_train.drop(columns=["datetime", "kWh"])
X_val = df_val.drop(columns=["datetime", "kWh"])

y_train = df_train["kWh"]
y_val = df_val["kWh"]

In [217]:
fitted_preprocessor = preprocessor.fit(X_train)

X_train_preprocessed = pd.DataFrame(
    fitted_preprocessor.transform(X_train),
    columns=fitted_preprocessor.get_feature_names_out(),
)

X_val_preprocessed = pd.DataFrame(
    fitted_preprocessor.transform(X_val),
    columns=fitted_preprocessor.get_feature_names_out(),
)

In [218]:
df_train_preprocessed = pd.concat(
    [df_train.filter(["datetime", "kWh"]), X_train_preprocessed], axis=1
)

df_val_preprocessed = pd.concat(
    [df_val.filter(["datetime", "kWh"]), X_val_preprocessed], axis=1
)

In [219]:
df_trainval = pd.concat(
    [df_train_preprocessed, df_val_preprocessed], axis=0
).reset_index(drop=True)


df_trainval.shape

(17520, 26)

In [220]:
(df_trainval["datetime"] - df_trainval["datetime"].shift(1)).value_counts()

datetime
0 days 01:00:00    17519
Name: count, dtype: int64

## Exogenous Variables

Define parameters:
- h: prediction horizon, here 24 (how far should be predicted into the future at once)
- l: lookback parameter (how many past observations should be considered), here 168 (one week)
    - Y lags
    - X lags

Start with exogenous variables due to edge case:
- at the start of the dataframe, there are no past observations to consider (need to be dropped)
- simplifying assumption: l can only be a multiple of h, so there doesn't need to be rounding applied to come back to the proper setup of predicting at midnight for the next day

In [248]:
h = 24
l = 7 * h
d = len(df_trainval.drop(columns=["datetime", "kWh"]).columns)
slice_points = np.arange(l, df_trainval.shape[0], h)

In [222]:
# y_target size: future y values
# X_target size: past y values + past X values + future X values
y = np.empty((0, 24))
X = np.empty((0, l + l * d + h * d))

for i, origin in enumerate(tqdm(slice_points)):
    # Create splits based on origin
    past_range = df_trainval.iloc[origin - l : origin]
    future_range = df_trainval.iloc[origin : origin + h]

    # Extract chunks of information
    y_future = future_range["kWh"].to_numpy()
    y_lags = past_range["kWh"].to_numpy()

    X_future = future_range.drop(columns=["datetime", "kWh"]).to_numpy().flatten()
    X_lags = past_range.drop(columns=["datetime", "kWh"]).to_numpy().flatten()

    # Append the information to the existing arrays
    y = np.vstack([y, y_future])
    X = np.vstack([X, np.hstack([X_future, X_lags, y_lags])])

  0%|          | 0/723 [00:00<?, ?it/s]

In [223]:
train_size = df_train_preprocessed.shape[0] - l
val_size = df_val_preprocessed.shape[0]

In [224]:
train_size

8592

In [225]:
val_size

8760

In [226]:
X_train = X[: int(train_size / h)]
y_train = y[: int(train_size / h)]

X_val = X[int(train_size / h) : int(train_size / h) + int(val_size / h)]
y_val = y[int(train_size / h) : int(train_size / h) + int(val_size / h)]

In [227]:
len(y_val.flatten())

8760

## Create function for tuning for aggregation

In [249]:
def aggregate_df(df, train_size, val_size, test_size, l=168, h=24):
    assert (
        l % h == 0
    ), f"Warning: 'l' ({l}) is not a multiple of 'h' ({h}). The aggregation may not work as expected."

    # Adjust train size for edge case of no historical data at start of data frame
    train_size = train_size - l
    d = len(df.drop(columns=["datetime", "kWh"]).columns)

    slice_points = np.arange(l, df.shape[0], h)

    # y_target size: future y values
    # X_target size: past y values + past X values + future X values
    y = np.empty((0, 24))
    X = np.empty((0, l + l * d + h * d))

    for i, origin in enumerate(slice_points):
        # Create splits based on origin
        past_range = df.iloc[origin - l : origin]
        future_range = df.iloc[origin : origin + h]

        # Extract chunks of information
        y_future = future_range["kWh"].to_numpy()
        y_lags = past_range["kWh"].to_numpy()

        X_future = future_range.drop(columns=["datetime", "kWh"]).to_numpy().flatten()
        X_lags = past_range.drop(columns=["datetime", "kWh"]).to_numpy().flatten()

        # Append the information to the existing arrays
        y = np.vstack([y, y_future])
        X = np.vstack([X, np.hstack([X_future, X_lags, y_lags])])

    # Recreate splits
    if train_size > 0:
        X_train = X[: int(train_size / h)]
        y_train = y[: int(train_size / h)]
    else:
        X_train = None
        y_train = None

    # Handle validation size being zero
    if val_size > 0:
        X_val = X[int(train_size / h) : int(train_size / h) + int(val_size / h)]
        y_val = y[int(train_size / h) : int(train_size / h) + int(val_size / h)]
    else:
        X_val = None
        y_val = None

    # Handle test size being zero
    if test_size > 0:
        X_test = X[-int(test_size / h) :]
        y_test = y[-int(test_size / h) :]
    else:
        X_test = None
        y_test = None

    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [250]:
X_train, X_val, X_test, y_train, y_val, y_test = aggregate_df(
    df=df_trainval,
    train_size=df_train_preprocessed.shape[0],
    val_size=df_val_preprocessed.shape[0],
    test_size=0,
    l=168,
    h=24,
)

# Hyperparameter Tuning

## Manual test

In [271]:
# Define the learning rate schedule
initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=100,
    decay_rate=0.98,
    staircase=True,
)


def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))


# Define the model
model = Sequential(
    [
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(24),
    ]
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss=rmse,
    metrics=[RootMeanSquaredError()],
)

# Set up Tensorboard
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Set up callbacks
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=100, restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping],
    verbose=0,
)

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss=rmse,
    metrics=[RootMeanSquaredError()],
)

# Set up Tensorboard
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Set up callbacks
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=100, restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=2000,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping],
    verbose=0,
)

In [256]:
val_preds = pd.DataFrame(
    {
        "datetime": df_val_preprocessed["datetime"],
        "pred": model.predict(X_val).flatten(),
        "kWh": y_val.flatten(),
    }
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [257]:
loss = root_mean_squared_error(y_pred=val_preds["pred"], y_true=val_preds["kWh"])
loss

256095.7007710484

In [261]:
# Create the figure with step lines for both actual and predicted values
fig = px.line(
    val_preds,
    x="datetime",
    y=["kWh", "pred"],
    labels={
        "datetime": "Date",
        "value": "Energy Consumption (kWh)",
        "variable": "Series",
    },
    title="Validation Fit",
    line_shape="hv",  # Set line shape to horizontal-vertical for step chart
)

# Customize the layout
fig.update_layout(
    template="plotly_white",
    legend=dict(title=""),
    xaxis_title="Date",
    yaxis_title="Energy Consumption (kWh)",
)

# Show the figure
fig.show()

In [260]:
# Need to account for the edge case of missing historical data at the start:
# Remove the first l observations from the datetime col that haven't been used
train_preds = pd.DataFrame(
    {
        "datetime": df_train_preprocessed["datetime"].iloc[
            df_train.shape[0] - y_train.shape[0] * y_train.shape[1] :
        ],
        "pred": model.predict(X_train).flatten(),
        "kWh": y_train.flatten(),
    }
)


# Create the figure with step lines for both actual and predicted values
fig = px.line(
    train_preds,
    x="datetime",
    y=["kWh", "pred"],
    labels={
        "datetime": "Date",
        "value": "Energy Consumption (kWh)",
        "variable": "Series",
    },
    title="Training Fit",
    line_shape="hv",  # Set line shape to horizontal-vertical for step chart
)

# Customize the layout
fig.update_layout(
    template="plotly_white",
    legend=dict(title=""),
    xaxis_title="Date",
    yaxis_title="Energy Consumption (kWh)",
)

# Show the figure
fig.show()

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [ ]:
config = {
    "initial_learning_rate": 0.001,  # Typical initial learning rate
    "decay_steps": 100,  # Decay every 100 steps
    "decay_rate": 0.95,  # Gradual learning rate decay
    "num_layers": 4,  # Moderate number of layers
    "units_per_layer": 64,  # Hidden units in each layer
    "activation": "ReLU",  # Activation function
    "h": 24,  # Output size, matching the prediction horizon
    "epochs": 50,  # Number of training epochs
    "batch_size": 32,  # Batch size
}

In [ ]:
# Define the learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=config["initial_learning_rate"],
    decay_steps=config["decay_steps"],
    decay_rate=config["decay_rate"],
    staircase=True,
)

# Build the model dynamically based on num_layers and units_per_layer
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))

if config["activation"] == "ReLU":
    for _ in range(config["num_layers"]):
        model.add(Dense(config["units_per_layer"]))
        model.add(ReLU())
elif config["activation"] == "LeakyReLU":
    for _ in range(config["num_layers"]):
        model.add(Dense(config["units_per_layer"]))
        model.add(LeakyReLU())

model.add(Dense(config["h"]))

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss=rmse,
    metrics=[RootMeanSquaredError()],
)

# Early stopping
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=10,  # Reduced patience for quicker debugging
    restore_best_weights=True,
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=config["epochs"],
    batch_size=config["batch_size"],
    callbacks=[early_stopping],
    verbose=1,
)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 5838541.0000 - root_mean_squared_error: 5874881.0000 - val_loss: 3166433.7500 - val_root_mean_squared_error: 3187433.5000
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2689736.5000 - root_mean_squared_error: 2739720.2500 - val_loss: 1032468.7500 - val_root_mean_squared_error: 1038668.4375
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 900851.0000 - root_mean_squared_error: 908668.7500 - val_loss: 712174.9375 - val_root_mean_squared_error: 715401.0625
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 701704.0625 - root_mean_squared_error: 703660.8750 - val_loss: 643622.6875 - val_root_mean_squared_error: 647741.0625
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 663830.5625 - root_mean_squared_error: 665939.8125 - val_loss: 640566.7500 - val_root_mean_squared_error: 644224.1250
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 718571.1875 - root_mean_squared_error: 721441.5000 -

## RayTune Tuning Loop

In [289]:
def keras_trainable(config):
    # try:
    # Define the learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate=config["initial_learning_rate"],
        decay_steps=config["decay_steps"],
        decay_rate=config["decay_rate"],
        staircase=True,
    )

    # Build the model dynamically based on num_layers and units_per_layer
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))

    if config["activation"] == "ReLU":
        for _ in range(config["num_layers"]):
            model.add(Dense(config["units_per_layer"]))
            model.add(ReLU())
    elif config["activation"] == "LeakyReLU":
        for _ in range(config["num_layers"]):
            model.add(Dense(config["units_per_layer"]))
            model.add(LeakyReLU())

    model.add(Dense(config["h"]))

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=lr_schedule),
        loss=rmse,
        metrics=[RootMeanSquaredError()],
    )

    # Early stopping
    early_stopping = EarlyStopping(
        monitor="val_loss",
        patience=50,
        restore_best_weights=True,
    )

    # Train the model
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=config["epochs"],
        batch_size=config["batch_size"],
        callbacks=[early_stopping],
        verbose=0,
    )

    # Report validation loss to Ray Tune
    train.report({"loss": min(history.history["val_loss"])})

    # except Exception as e:
    #     train.report({"loss": float("inf")})

In [291]:
# Need this line for locally defined modules to work with ray
# ray.init(runtime_env={"working_dir": "."}, ignore_reinit_error=True)
np.random.seed(42)

analysis = tune.run(
    keras_trainable,
    config={
        "initial_learning_rate": tune.loguniform(1e-5, 1e-2),
        "decay_steps": tune.choice([10, 25, 50, 100, 200]),
        "decay_rate": tune.uniform(0.8, 0.99),
        "num_layers": tune.randint(1, 5 + 1),  # Max 5 layers
        "units_per_layer": tune.choice([32, 64, 128, 256, 512]),
        "activation": tune.choice(["ReLU", "LeakyReLU"]),
        "batch_size": tune.choice([16, 32, 64, 128]),
        "epochs": 2000,
        "h": 24,
    },
    metric="loss",
    mode="min",
    name="DNN",
    search_alg=OptunaSearch(),
    time_budget_s=60 * 60 * 9,
    num_samples=-1,
    max_concurrent_trials=8,
    raise_on_failed_trial=False,
    trial_dirname_creator=lambda trial: f"{trial.trainable_name}_{trial.trial_id}",
)

2024-11-20 00:14:34,174	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-11-20 00:14:34,365] A new study created in memory with name: optuna


Trial name,loss
keras_trainable_49f59c81,474362
keras_trainable_7de7f650,374975


2024-11-20 00:18:37,785	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-11-20 00:18:37,873	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/mathi/ray_results/DNN' in 0.0879s.
2024-11-20 00:18:48,864	INFO tune.py:1041 -- Total run time: 254.69 seconds (243.22 seconds for the tuning loop).
2024-11-20 00:18:48,866	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-11-20 00:18:48,907	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 1 trial(s):
- keras_trainable_8c3c4d24: FileNotFoundError('Could not fetch metrics for keras_trainable_8c3c4d24: both result.json and progress.csv were not found at C:/Users/mathi/ray_results/DNN/keras_trainable_8c3c4d24')


In [15]:
analysis.dataframe().to_csv("2_SARIMAX_trials.csv", index=False)

NameError: name 'analysis' is not defined

# Model Evaluation